# Modele

Przyglądając się niższemu poziomowi API Transformers — modelom, które opakowują kod PyTorch dla samych transformatorów.

Ten notebook może działać na niedrogim lub darmowym środowisku uruchomieniowym T4.

In [1]:
# Instalujemy wymagane biblioteki, używając menedżera pakietów pip.
# Opcja "-q" (quiet) sprawia, że instalacja nie wyświetla zbędnych komunikatów.

'''	•	requests – obsługa żądań HTTP, przydatna np. do pobierania modeli z Hugging Face.
	•	torch – główna biblioteka PyTorch, służąca do trenowania i uruchamiania modeli neuronowych.
	•	bitsandbytes – pozwala na efektywne wykorzystanie pamięci GPU, kluczowe w przypadku dużych modeli AI.
	•	transformers – biblioteka Hugging Face do pracy z modelami Transformer.
	•	sentencepiece – tokenizer działający niezależnie od języka (często stosowany w modelach wielojęzycznych).
	•	accelerate – ułatwia uruchamianie modeli na różnych urządzeniach (CPU, GPU, TPU).'''

!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.5 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

# Importujemy funkcję `login` z `huggingface_hub`, która pozwala na uwierzytelnienie użytkownika
# w serwisie Hugging Face. Dzięki temu możemy pobierać modele wymagające autoryzacji.
from huggingface_hub import login

# Importujemy klasy z biblioteki `transformers`, które pozwalają na pracę z modelami NLP:
# - `AutoTokenizer` - automatycznie ładuje odpowiedni tokenizer dla wybranego modelu.
# - `AutoModelForCausalLM` - automatycznie ładuje model do generowania tekstu (Causal Language Model).
# - `TextStreamer` - umożliwia strumieniowe generowanie tekstu, co jest przydatne w aplikacjach interaktywnych.
# - `BitsAndBytesConfig` - konfiguracja optymalizacji pamięci dla modeli za pomocą biblioteki bitsandbytes.
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig

# Importujemy bibliotekę `torch`, która jest podstawą frameworka PyTorch, używanego do uruchamiania modeli AI.
import torch

# Sign in to Hugging Face

1. If you haven't already done so, create a free HuggingFace account at https://huggingface.co and navigate to Settings, then Create a new API token, giving yourself write permissions by clicking on the WRITE tab

2. Press the "key" icon on the side panel to the left, and add a new secret:
`HF_TOKEN = your_token`

3. Execute the cell below to log in.

In [3]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [5]:
# Definiujemy nazwy dostępnych modeli instruktażowych do generowania tekstu.
# Modele te zostały wytrenowane do przestrzegania instrukcji podanych w promptach.

# Model LLaMA 3.1 Instruct od Meta - flagowy model Meta przeznaczony do przetwarzania języka naturalnego.
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Model Phi-3 Mini 4k Instruct od Microsoft - zoptymalizowany pod kątem wydajności i mniejszego zużycia zasobów.
PHI3 = "microsoft/Phi-3-mini-4k-instruct"

# Model Gemma 2 IT od Google - mniejsza wersja modelu Gemini, przeznaczona do inferencji instruktażowej.
GEMMA2 = "google/gemma-2-2b-it"

# Model Qwen2 7B Instruct od Alibaba (Qwen) - model przeznaczony jako ćwiczenie dla użytkownika.
QWEN2 = "Qwen/Qwen2-7B-Instruct"  # Zadanie dla Ciebie - sprawdź, jak działa ten model!

# Model Mixtral 8x7B Instruct od Mistral - zaawansowany model typu Mixture of Experts.
# Uwaga: Może nie zmieścić się w pamięci Twojego GPU, więc w razie problemów warto spróbować innych modeli.
MIXTRAL = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [6]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Tell a light-hearted joke for a room of Data Scientists"}
  ]

# Accessing Llama 3.1 from Meta

In order to use the fantastic Llama 3.1, Meta does require you to sign their terms of service.

Visit their model instructions page in Hugging Face:
https://huggingface.co/meta-llama/Meta-Llama-3.1-8B

At the top of the page are instructions on how to agree to their terms. If possible, you should use the same email as your huggingface account.

In my experience approval comes in a couple of minutes. Once you've been approved for any 3.1 model, it applies to the whole family of models.

In [7]:
# Konfiguracja kwantyzacji - pozwala załadować model do pamięci, jednocześnie redukując jej zużycie

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Włącza kwantyzację 4-bitową, co znacząco zmniejsza zapotrzebowanie na pamięć GPU
    bnb_4bit_use_double_quant=True,  # Aktywuje podwójną kwantyzację dla jeszcze większej efektywności
    bnb_4bit_compute_dtype=torch.bfloat16,  # Ustawia precyzję obliczeń na bfloat16 (lepsza wydajność na nowoczesnych GPU)
    bnb_4bit_quant_type="nf4"  # Używa specyficznej metody kwantyzacji NF4 (Normal Float 4) dla lepszej dokładności
)

Jeśli następna komórka wyświetla błąd uprawnień 403, sprawdź:
1. Czy jesteś zalogowany do HuggingFace? Spróbuj uruchomić `login()`, aby sprawdzić, czy klucz działa
2. Czy skonfigurowałeś klucz API z pełnymi uprawnieniami do odczytu i zapisu?
3. Jeśli odwiedzisz stronę Llama3.1 pod adresem https://huggingface.co/meta-llama/Meta-Llama-3.1-8B, czy pokazuje się, że masz dostęp do modelu w pobliżu góry?


In [8]:
# Ładowanie tokenizera dla modelu LLAMA
tokenizer = AutoTokenizer.from_pretrained(LLAMA)

# Ustawienie tokena paddingu (wyrównania) na token końca sekwencji (eos)
# Jest to potrzebne, gdy model nie ma jawnie zdefiniowanego tokena paddingu
tokenizer.pad_token = tokenizer.eos_token

# Konwersja wiadomości (chat history) do formatu tokenów zgodnego z modelem
# "apply_chat_template" formatuje dane wejściowe zgodnie ze schematem czatu
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
# Zamiana tokenów na tensor PyTorch i przeniesienie na GPU dla szybszych obliczeń

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [9]:
# Ładowanie modelu językowego (LLM) w trybie generacji tekstu
model = AutoModelForCausalLM.from_pretrained(
    LLAMA,                # Nazwa modelu do załadowania (LLAMA 3.1)
    device_map="auto",    # Automatyczne przypisanie modelu do dostępnego sprzętu (CPU/GPU)
    quantization_config=quant_config  # Konfiguracja kwantyzacji (redukcja zużycia pamięci)
)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [10]:
# Obliczanie zużycia pamięci przez załadowany model
memory = model.get_memory_footprint() / 1e6  # Konwersja bajtów na megabajty (MB)

# Wyświetlenie wyników w czytelnej formie
print(f"Memory footprint: {memory:,.1f} MB")  # Formatowanie z separatorami tysięcznymi i 1 miejscem po przecinku

Memory footprint: 5,591.5 MB


In [11]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((409

In [12]:
# 'model.generate' generuje dane na podstawie wejściowych danych 'inputs' i ustawia maksymalną liczbę nowych tokenów, które mają zostać wygenerowane na 80.
outputs = model.generate(inputs, max_new_tokens=80)

# 'tokenizer.decode' dekoduje wygenerowane dane (tokeny) z formatu numerycznego na czytelny tekst. 'outputs[0]' odnosi się do pierwszego wyniku w zbiorze wyników.
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell a light-hearted joke for a room of Data Scientists<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Why did the logistic regression model go to therapy?

Because it was struggling to classify its emotions.<|eot_id|>


In [13]:
# Clean up

del inputs, outputs, model
torch.cuda.empty_cache()

Używam narzędzia HuggingFace o nazwie TextStreamer, aby wyniki były przesyłane strumieniowo.
Aby przesyłać strumieniowo wyniki, po prostu zastępujemy:
`outputs = model.generate(inputs, max_new_tokens=80)`
Na:
`streamer = TextStreamer(tokenizer)`
`outputs = model.generate(inputs, max_new_tokens=80, streamer=streamer)`

Dodałem również argument `add_generation_prompt=True` do mojego wywołania, aby utworzyć szablon czatu. Dzięki temu Phi generuje odpowiedź na pytanie, zamiast po prostu przewidywać, jak będzie kontynuowany monit użytkownika. Spróbuj poeksperymentować z ustawieniem tego na False, aby zobaczyć, co się stanie. Możesz przeczytać o tym argumencie tutaj:

https://huggingface.co/docs/transformers/main/en/chat_templating#what-are-generation-prompts

Dziękuję studentowi Piotrowi B za poruszenie tego tematu!

In [14]:
# Wrapping everything in a function - and adding Streaming and generation prompts

def generate(model, messages):
  tokenizer = AutoTokenizer.from_pretrained(model)
  tokenizer.pad_token = tokenizer.eos_token
  inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")
  streamer = TextStreamer(tokenizer)
  model = AutoModelForCausalLM.from_pretrained(model, device_map="auto", quantization_config=quant_config)
  outputs = model.generate(inputs, max_new_tokens=80, streamer=streamer)
  del tokenizer, streamer, model, inputs, outputs
  torch.cuda.empty_cache()

In [15]:
generate(PHI3, messages)

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

<|system|> You are a helpful assistant<|end|><|user|> Tell a light-hearted joke for a room of Data Scientists<|end|><|assistant|> Why did the data scientist break up with the algorithm?

Because it was always calculating the wrong variables in their relationship!<|end|>


## Dostęp do Gemma z Google

Pewien student poinformował mnie (dziękuję, Alex K!), że Google wymaga teraz również zaakceptowania ich warunków w HuggingFace przed użyciem Gemma.

Odwiedź ich stronę modelu pod tym linkiem i potwierdź, że akceptujesz ich warunki, aby uzyskać dostęp.

https://huggingface.co/google/gemma-2-2b-it

In [ ]:
messages = [
    {"role": "user", "content": "Tell a light-hearted joke for a room of Data Scientists"}
  ]
generate(GEMMA2, messages)